In [26]:
import re
import PyPDF2
from PIL import Image
import pytesseract

In [54]:

path_to_cv_1 = "data/cv-example-1.png"
path_to_cv_2 = "data/cv-example-2.pdf"
path_to_cv_3 = "data/cv-example-3.pdf"

In [32]:
def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text

def extract_text_from_image(image_path):
    return pytesseract.image_to_string(Image.open(image_path))


In [56]:
# text = extract_text_from_pdf(path_to_cv_3)
# print(text)

text = extract_text_from_image(path_to_cv_1)
print(text)

MARK RICE

PROFESSIONAL SUMMARY

Energetic Primary School Teacher driven to continue to learn and grow
professionally to be the most effective teacher. Able to understand and
implement new technologies and teaching strategies. Excel at working

with different learning styles.

WORK HISTORY

Primary Teacher, 05/2019 to Current
Edinburgh Boys School - Edinburgh, Scotland

+ Established strong, supportive relationships with families, encouraging
parental involvement to boost children's achievement.

+ Integrated creative learning strategies, such as games and
team-building exercises, resulting in increased classroom engagement.

+ Evaluated students' academic progress, liaising with parents to
facilitate cohesive learning environments at home and in school.

+ Utilised visual aids and classroom displays to support engaging lesson
delivery and enhance student understanding.

Primary Teacher, 01/2014 to 04/2019

St Christopher's School - Dunbar , East Lothian

+ Prepared and delivered creat

In [83]:
# get first line to get name
name = text.split("\n")[0].strip().lower()

print(name)

mark rice


In [93]:
' '.join("mark rice smith".split()[1:])

'rice smith'

In [58]:
# regex to extract email
email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
emails = re.findall(email_pattern, text)
print(emails)

[]


In [94]:
phone_pattern = r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
phones = re.findall(phone_pattern, text)
print(phones)


['0791234567']


In [60]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for entity in doc.ents:
    print(entity.label_, entity.text)

PERSON MARK RICE
ORG Energetic Primary School
PERSON Primary Teacher
PERSON 05/2019
PERSON Current
Edinburgh Boys School - Edinburgh
GPE Scotland
PERSON Primary Teacher
PERSON St Christopher's
ORG School - Dunbar
NORP East Lothian
WORK_OF_ART + Prepared
DATE 5-11
LANGUAGE English
ORG SEND
NORP + Marked
PERSON Primary Teacher
CARDINAL 10/2009 to 12/2013
ORG St Matthews Primary - Edinburgh
GPE Scotland
CARDINAL 2
ORG + Created
DATE daily
CARDINAL 52
PERSON Primary
DATE 5-11
DATE 2009
ORG University of Edinburgh
GPE Edinburgh
LANGUAGE English
DATE 2008
ORG University of Strathclyde - Glasgow


In [61]:
for ent in doc.ents:
    if ent.label_ == "ORG" or ent.label_ == "WORK_OF_ART":
        print(ent.text)

Energetic Primary School
School - Dunbar
+ Prepared
SEND
St Matthews Primary - Edinburgh
+ Created
University of Edinburgh
University of Strathclyde - Glasgow


In [62]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import os

# bypass VPN to connect to hugging face hub
os.environ['CURL_CA_BUNDLE'] = ''

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

ner_results = nlp(text)
print(ner_results)


/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'entity': 'B-ORG', 'score': 0.5830573, 'index': 78, 'word': 'Edinburgh', 'start': 328, 'end': 337}, {'entity': 'I-ORG', 'score': 0.8025946, 'index': 79, 'word': 'Boys', 'start': 338, 'end': 342}, {'entity': 'I-ORG', 'score': 0.6076678, 'index': 80, 'word': 'School', 'start': 343, 'end': 349}, {'entity': 'B-LOC', 'score': 0.99685425, 'index': 82, 'word': 'Edinburgh', 'start': 352, 'end': 361}, {'entity': 'B-LOC', 'score': 0.9994856, 'index': 84, 'word': 'Scotland', 'start': 363, 'end': 371}, {'entity': 'B-ORG', 'score': 0.61656016, 'index': 180, 'word': 'St', 'start': 921, 'end': 923}, {'entity': 'I-LOC', 'score': 0.72082955, 'index': 181, 'word': 'Christopher', 'start': 924, 'end': 935}, {'entity': 'I-ORG', 'score': 0.49424717, 'index': 182, 'word': "'", 'start': 935, 'end': 936}, {'entity': 'I-ORG', 'score': 0.5858895, 'index': 183, 'word': 's', 'start': 936, 'end': 937}, {'entity': 'I-ORG', 'score': 0.4963573, 'index': 184, 'word': 'School', 'start': 938, 'end': 944}, {'entity': 'B

In [64]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/tinyroberta-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)

/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [67]:
QA_input = {
    'question': 'What is my name?',
    'context': text
}
res = nlp(QA_input)
print(res)
# b) Load model & tokenizer
# model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)


{'score': 7.837845927838316e-09, 'start': 0, 'end': 9, 'answer': 'MARK RICE'}


In [68]:
QA_input = {
    'question': 'What job titles have I had?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 2.4087992045451756e-08, 'start': 1338, 'end': 1353, 'answer': 'Primary Teacher'}


In [71]:
QA_input = {
    'question': 'What university or school did I attend?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 1.250165132660186e-05, 'start': 2074, 'end': 2097, 'answer': 'University of Edinburgh'}


In [74]:
QA_input = {
    'question': 'What degree type did I study at university?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 4.330485353420954e-06, 'start': 2111, 'end': 2127, 'answer': 'Bachelor of Arts'}


In [76]:
QA_input = {
    'question': 'What subject did I study at the University of Edinburgh?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 1.8227426323846885e-07, 'start': 2129, 'end': 2147, 'answer': 'English\nLiterature'}


In [77]:
QA_input = {
    'question': 'What is my phone number?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 0.6240267753601074, 'start': 1778, 'end': 1789, 'answer': '07912345678'}


In [96]:
QA_input = {
    'question': 'What is my email?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 0.00036871983320452273, 'start': 1798, 'end': 1815, 'answer': 'mark.rice@example'}


In [82]:
QA_input = {
    'question': 'What is my work history?',
    'context': text
}
res = nlp(QA_input)
print(res)

{'score': 0.0006440834840759635, 'start': 291, 'end': 307, 'answer': '\nPrimary Teacher'}


# Other thoughts

- job title, education - pre-defined lists, NER, fine-tuned NER model, NER to subset options and then match to list, LLMs Q/A
- job experience - extract text with regex after extracting job title
- hobbies, skills - keyword list